In [1]:
import os
import os.path as path
import gc
import re
import math
import json
import random

In [2]:
from datetime import datetime
import time

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_row', 50)
pd.set_option('display.max_columns', 100)

In [4]:
# 디렉토리 기본 경로 설정
DIR_PATH = path.join('..')
DIR_SAVE_PATH = path.join('.')

print(DIR_PATH)
print(DIR_SAVE_PATH)

..
.


In [5]:
# sql_bean.csv
bean_read = pd.read_csv(path.join('.', 'sql_bean.csv'), low_memory=False, encoding='utf-8')

print(bean_read.shape)
bean_read.head()

(47, 8)


,idx,created_date,updated_date,name_ko,name_en,summary,thumbnail,user_grade
0,1,2023-03-01 12:00:00.000000,NaN,에티오피아 예가체프 G2 워시드 (중배전),Ethiopia Yega Chef G2,"산미와 꽃 향기가 조화를 이루며, 원두의 맛을 최대한 살리기 위해 노력합니다.",default_bean.png,0
1,2,2023-03-01 12:43:12.000000,NaN,과테말라 안티구아 SHB 워시드 (강배전),Guatemala Antigua SHB,"한국인들은 단맛을 좋아해서 부드럽고 달콤한 커피를 선호하며, 지역의 풍토가 커피 맛...",default_bean.png,0
2,3,2023-03-01 13:26:24.000000,NaN,에티오피아 코케 G1 펄프드내추럴 (중배전),Ethiopia Koke G1,"산미(신맛)이 높고 향이 풍부하며, 반대로 바디감과 쓴맛은 적습니다. 또한 허니 프...",default_bean.png,0
3,4,2023-03-01 14:09:36.000000,NaN,케냐 키암부 AA 워시드 (중배전),Kenya Kyambu AA,"산미(신맛)가 적은 커피이며, 클린한 바디감의 고소함이 좋습니다. 고급스러운 고소함...",default_bean.png,0
4,5,2023-03-01 14:52:48.000000,NaN,콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),Colombia Colombian SUPREMO,부드럽고 마일드한 맛과 깊은향으로 누구나 쉽게 즐길수 있는 커피입니다. 메이플시럽의...,default_bean.png,0


In [6]:
# sql_capsule.csv
capsule_read = pd.read_csv(path.join('.', 'sql_capsule.csv'), low_memory=False, encoding='utf-8')

print(capsule_read.shape)
capsule_read.head()

(152, 8)


,idx,created_date,updated_date,name_en,name_ko,summary,thumbnail,user_grade
0,1,2023-03-31 12:00:00,NaN,Amaretti,아마레티,"아몬드, 비스킷",default_capsule.png,0
1,2,2023-03-31 12:00:00,NaN,Bukeela ka Ethiopia,부케엘라 카 에티오피아,"과일, 플로럴",default_capsule.png,0
2,3,2023-03-31 12:00:00,NaN,Cafe Istanbul,카페 이스탄불,"강렬한, 자극적인",default_capsule.png,0
3,4,2023-03-31 12:00:00,NaN,Caramelito,카라멜리또,카라멜,default_capsule.png,0
4,5,2023-03-31 12:00:00,NaN,Ciocattino,초콜라티노,초코,default_capsule.png,0


In [7]:
# sql_member.csv
member_read = pd.read_csv(path.join('.', 'sql_member.csv'), low_memory=False, encoding='utf-8')

print(member_read.shape)
member_read.head()

(1000, 12)


,idx,created_date,updated_date,age_range,expired,gender,hashcode,member_email,nickname,profileImg,role,sns_type
0,1,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,20~29,N,male,N36WH,kwonjiwoo8689@naver.com,권지우,http://k.kakaocdn.net/dn/dpk9l1/btqmGhA2lKL/Oz...,ROLE_MEMBER,KAKAO
1,2,2023-03-01 12:43:12.000000,2023-03-01 12:43:12.000000,20~29,N,male,5P868,jusang6794@gmail.com,홍주상,http://k.kakaocdn.net/dn/dpk9l1/btqmGhA2lKL/Oz...,ROLE_MEMBER,KAKAO
2,3,2023-03-01 13:26:24.000000,2023-03-01 13:26:24.000000,20~29,N,female,9MRTC,ohe9819@naver.com,오하은,http://k.kakaocdn.net/dn/dpk9l1/btqmGhA2lKL/Oz...,ROLE_MEMBER,KAKAO
3,4,2023-03-01 14:09:36.000000,2023-03-01 14:09:36.000000,20~29,N,male,MSNT1,kwonhahwan4662@gmail.com,권하환,http://k.kakaocdn.net/dn/dpk9l1/btqmGhA2lKL/Oz...,ROLE_MEMBER,KAKAO
4,5,2023-03-01 14:52:48.000000,2023-03-01 14:52:48.000000,50~59,N,female,VZ6XF,dongji9704@naver.com,임동지,http://k.kakaocdn.net/dn/dpk9l1/btqmGhA2lKL/Oz...,ROLE_MEMBER,KAKAO


In [8]:
print(bean_read.shape[0])
print(capsule_read.shape[0])

47
152


In [9]:
def generate_random_like(item_len):
    random_list = [random.choices([1, np.NaN], weights=[item_len * 0.2, item_len * 0.8])[0] for i in range(item_len)]
    random_series = pd.Series(random_list, index=[i+1 for i in range(item_len)])
    
    return random_series

In [10]:
generate_random_like(10)

1     NaN
2     1.0
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     1.0
9     NaN
10    NaN
dtype: float64

In [11]:
like_bean_df = pd.DataFrame(index=member_read['idx'], columns=bean_read['idx'])
like_bean_df.index.rename('member_idx', inplace=True)
like_bean_df.columns.rename('bean_idx', inplace=True)
print(like_bean_df.shape)
like_bean_df.head()

(1000, 47)


bean_idx,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
member_idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
like_bean_df = like_bean_df.apply(lambda x: generate_random_like(like_bean_df.shape[1]), axis=1)
like_bean_df.columns.rename('item_idx', inplace=True)
like_bean_df.head()

item_idx,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
member_idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
like_bean_df = like_bean_df.melt(ignore_index=False).reset_index()
like_bean_df.dropna(inplace=True)
like_bean_df.reset_index(drop=True, inplace=True)
like_bean_df.drop(['value'], axis=1, inplace=True)
like_bean_df['created_date'] = member_read['created_date'].iloc[0]
like_bean_df['updated_date'] = member_read['created_date'].iloc[0]
like_bean_df['expired'] = 'N'
like_bean_df['item_type'] = 'bean'
like_bean_df['idx'] = like_bean_df.index + 1
like_bean_df = like_bean_df[['idx', 'created_date', 'updated_date', 'expired', 'item_idx', 'item_type', 'member_idx']]
like_bean_df

,idx,created_date,updated_date,expired,item_idx,item_type,member_idx
0,1,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,1,bean,4
1,2,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,1,bean,7
2,3,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,1,bean,23
3,4,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,1,bean,29
4,5,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,1,bean,46
...,...,...,...,...,...,...,...
9374,9375,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,47,bean,957
9375,9376,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,47,bean,960
9376,9377,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,47,bean,974
9377,9378,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,47,bean,991


In [14]:
like_capsule_df = pd.DataFrame(index=member_read['idx'], columns=capsule_read['idx'])
like_capsule_df.index.rename('member_idx', inplace=True)
like_capsule_df.columns.rename('capsule_idx', inplace=True)
print(like_capsule_df.shape)
like_capsule_df.head()

(1000, 152)


capsule_idx,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,...,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152
member_idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
like_capsule_df = like_capsule_df.apply(lambda x: generate_random_like(like_capsule_df.shape[1]), axis=1)
like_capsule_df.columns.rename('item_idx', inplace=True)
like_capsule_df.head()

item_idx,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,...,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152
member_idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0
3,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0


In [16]:
like_capsule_df = like_capsule_df.melt(ignore_index=False).reset_index()
like_capsule_df.dropna(inplace=True)
like_capsule_df.reset_index(drop=True, inplace=True)
like_capsule_df.drop(['value'], axis=1, inplace=True)
like_capsule_df['created_date'] = member_read['created_date'].iloc[0]
like_capsule_df['updated_date'] = member_read['created_date'].iloc[0]
like_capsule_df['expired'] = 'N'
like_capsule_df['item_type'] = 'capsule'
like_capsule_df['idx'] = like_capsule_df.index + like_bean_df.shape[0] + 1
like_capsule_df = like_capsule_df[['idx', 'created_date', 'updated_date', 'expired', 'item_idx', 'item_type', 'member_idx']]
like_capsule_df

,idx,created_date,updated_date,expired,item_idx,item_type,member_idx
0,9380,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,1,capsule,4
1,9381,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,1,capsule,14
2,9382,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,1,capsule,21
3,9383,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,1,capsule,23
4,9384,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,1,capsule,25
...,...,...,...,...,...,...,...
30240,39620,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,152,capsule,972
30241,39621,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,152,capsule,974
30242,39622,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,152,capsule,992
30243,39623,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,152,capsule,998


In [17]:
like_list_df = pd.concat([like_bean_df, like_capsule_df], ignore_index=True)
like_list_df

,idx,created_date,updated_date,expired,item_idx,item_type,member_idx
0,1,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,1,bean,4
1,2,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,1,bean,7
2,3,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,1,bean,23
3,4,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,1,bean,29
4,5,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,1,bean,46
...,...,...,...,...,...,...,...
39619,39620,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,152,capsule,972
39620,39621,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,152,capsule,974
39621,39622,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,152,capsule,992
39622,39623,2023-03-01 12:00:00.000000,2023-03-01 12:00:00.000000,N,152,capsule,998


In [18]:
# 파일 저장
os.makedirs(DIR_SAVE_PATH, exist_ok=True)
like_list_df.to_csv(path.join(DIR_SAVE_PATH, 'sql_like_list_new.csv'), sep=',', index=False, encoding='utf-8')